In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import json
from distutils.dir_util import copy_tree

In [2]:
datasets=["axial","coronal","saggital"]
labels=["train-abnormal","train-acl","train-meniscus"]


In [3]:
if(not os.path.exists("Datasets")):
    os.mkdir("Datasets")

for l in labels:
    try:
        l_name=l.split("-")[1]
        os.mkdir("Datasets/"+l_name)

    except:
        print("Directory already exists")
    try:
        os.mkdir("Datasets/"+l_name+"/0/")
    except:
        pass
    try:
        os.mkdir("Datasets/"+l_name+"/1/")
    except:
        pass

Directory already exists
Directory already exists
Directory already exists


In [4]:
data_images={}
for d in datasets:
    data_images[d]=np.load(d+".npy")

In [5]:
label_data={}
for l in labels:
    l_dat=pd.read_csv(l+".csv",names=['img','label'])
    label_data[l]=l_dat

In [19]:
filler = np.zeros([256,256,3],dtype=np.uint8)
for i in range(data_images['axial'].shape[0]):
    ax=data_images['axial'][i]
    cor=data_images['coronal'][i]
    sag=data_images['saggital'][i]
    first_row=np.concatenate((ax,cor),axis=1)
    second_row=np.concatenate((sag,filler),axis=1)
    out_image=np.concatenate((first_row,second_row),axis=0)
    im=Image.fromarray(out_image)
    for l in labels:
        if(label_data[l]['label'].iloc[i]==1):
            im.save("Datasets/"+l.split("-")[1]+"/1/"+str(i)+".png")
        else:
            im.save("Datasets/"+l.split("-")[1]+"/0/"+str(i)+".png")
            
    print(i,label_data[l].iloc[i]['label']==1,"done!")          
    

0 False done!
1 True done!
2 False done!
3 True done!
4 False done!
5 True done!
6 False done!
7 False done!
8 False done!
9 False done!
10 True done!
11 False done!
12 False done!
13 False done!
14 False done!
15 False done!
16 True done!
17 False done!
18 True done!
19 True done!
20 True done!
21 False done!
22 True done!
23 False done!
24 False done!
25 False done!
26 False done!
27 True done!
28 True done!
29 False done!
30 True done!
31 False done!
32 True done!
33 True done!
34 False done!
35 True done!
36 True done!
37 True done!
38 True done!
39 True done!
40 False done!
41 False done!
42 True done!
43 True done!
44 False done!
45 True done!
46 False done!
47 False done!
48 False done!
49 True done!
50 False done!
51 False done!
52 True done!
53 False done!
54 True done!
55 False done!
56 False done!
57 True done!
58 False done!
59 True done!
60 False done!
61 False done!
62 False done!
63 True done!
64 False done!
65 False done!
66 False done!
67 True done!
68 False done!
69 F

In [6]:
for l in labels:
    l_name=l.split("-")[1]
    path="Datasets/"+l_name
    zeros=os.listdir(path+"/0")
    ones=os.listdir(path+"/1")
    zeros_class=[(l_name+"/0/"+x,0) for x in zeros]
    ones_class=[(l_name+"/1/"+x,1) for x in ones]
    out_class=zeros_class+ones_class
    out_class.sort(key=lambda x:float(x[0].split("/")[-1].split(".")[0]))
    out_class={"labels":out_class}
    json.dump(out_class,open(path+"/"+l_name+".json","w"))
    pass

# Balance the dataset!

In [8]:
original_folder="Datasets"
target_folder="Datasets_balanced"
copy_tree(original_folder, target_folder)
for l in labels:
    path="Datasets_balanced/"+l.split("-")[1]
    zeros=len(os.listdir(path+"/0"))
    ones=len(os.listdir(path+"/1"))
    np.random.seed(0) #seed for making sure everytime it runs delete the same images, you can change that
    if(zeros>ones):
        zero_dirs=os.listdir(path+"/0")
        to_be_deleted=np.random.choice(zero_dirs,zeros-ones,replace=False)
        for d in to_be_deleted:
            os.remove(path+"/0/"+d)
    elif(ones>zeros):
        one_dirs=os.listdir(path+"/1")
        to_be_deleted=np.random.choice(one_dirs,ones-zeros,replace=False)
        for d in to_be_deleted:
            os.remove(path+"/1/"+d)
    else:
        print("No need to delete")
        
for l in labels:
    l_name=l.split("-")[1]
    path=target_folder+"/"+l_name
    zeros=os.listdir(path+"/0")
    ones=os.listdir(path+"/1")
    zeros_class=[(l_name+"/0/"+x,0) for x in zeros]
    ones_class=[(l_name+"/1/"+x,1) for x in ones]
    out_class=zeros_class+ones_class
    out_class.sort(key=lambda x:float(x[0].split("/")[-1].split(".")[0]))
    out_class={"labels":out_class}
    json.dump(out_class,open(path+"/"+l_name+".json","w"))
    pass

#Merge Folders

In [11]:
folders=["Datasets","Datasets_balanced"]
for f in folders:
    for l in labels:
        l_name=l.split("-")[1]
        path=f+"/"+l_name
        zeros=os.listdir(path+"/0")
        ones=os.listdir(path+"/1")
        zeros_class=[(l_name+"/"+x,0) for x in zeros]
        ones_class=[(l_name+"/"+x,1) for x in ones]
        out_class=zeros_class+ones_class
        out_class.sort(key=lambda x:float(x[0].split("/")[-1].split(".")[0]))
        out_class={"labels":out_class}
        json.dump(out_class,open(path+"/"+l_name+".json","w"))
        
        copy_tree(path+"/0",f+"/"+l_name)
        copy_tree(path+"/1",f+"/"+l_name)
        #os.rmdir(path+"/0")#Optional uncomment them if you want to get rid of old folders
        #os.rmdir(path+"/1")
        pass


